<a href="https://datamics.com/de/courses/"><img src=../DATA/bg_datamics_top.png></a>

<em text-align:center>© Datamics</em>
# Regressionsübung - Lösungen

Kalifornien Housing Daten

Dieser Datensatz enthält Informationen über alle Blockgruppen in Kalifornien aus der Volkszählung von 1990. In dieser Stichprobe umfasst eine Blockgruppe durchschnittlich 1425,5 Personen, die in einem geografisch kompakten Gebiet leben. 

Die Aufgabe besteht darin, den mittleren Hauswert jedes Blocks aus den Werten der übrigen Variablen zu nähern. 

 Es wurde aus dem LIACC-Repository bezogen. Die ursprüngliche Seite, auf der sich der Datensatz befindet, ist: http://www.liaad.up.pt/~ltorgo/Regression/DataSets.html.
 




Die Features:
 
* housingMedianAge: kontinuierlich. 
* totalRooms: kontinuierlich. 
* totalBedrooms: kontinuierlich. 
* population: kontinuierlich. 
* households: kontinuierlich. 
* medianIncome: kontinuierlich. 
* medianHouseValue: kontinuierlich. 

## Die Daten

** Importiere die Datei cal_housing.csv mit Pandas. Trenne es in ein Trainings- (70%) und ein Testset (30%). **

In [1]:
import pandas as pd

In [2]:
housing = pd.read_csv('cal_housing_clean.csv')

FileNotFoundError: File b'cal_housing_clean.csv' does not exist

In [ ]:
housing.head()

In [ ]:
housing.describe().transpose()

In [ ]:
x_data = housing.drop(['medianHouseValue'],axis=1)

In [ ]:
y_val = housing['medianHouseValue']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x_data,y_val,test_size=0.3,random_state=101)

### Skalierung der Feature-Daten

** Verwende die Sklearn-Vorverarbeitung, um einen MinMaxScaler für die Feature-Daten zu erstellen. Passe diesen Scaler nur an die Trainingsdaten an. Dann verwendenst du es, um X_test und X_train zu transformieren. Verwende dann die skalierten X_test und X_train zusammen mit pd.Dataframe, um zwei Datenrahmen aus skalierten Daten neu zu erstellen. **


In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()

In [ ]:
scaler.fit(X_train)

In [ ]:
X_train = pd.DataFrame(data=scaler.transform(X_train),columns = X_train.columns,index=X_train.index)

In [ ]:
X_test = pd.DataFrame(data=scaler.transform(X_test),columns = X_test.columns,index=X_test.index)

### Feature Spalten erstellen

** Erstellen  die notwendigen tf.feature_column Objekte für den Schätzer. Sie sollten alle als fortlaufende numerische Spalten behandelt werden. **


In [ ]:
housing.columns

In [ ]:
import tensorflow as tf

In [ ]:
age = tf.feature_column.numeric_column('housingMedianAge')
rooms = tf.feature_column.numeric_column('totalRooms')
bedrooms = tf.feature_column.numeric_column('totalBedrooms')
pop = tf.feature_column.numeric_column('population')
households = tf.feature_column.numeric_column('households')
income = tf.feature_column.numeric_column('medianIncome')

In [ ]:
feat_cols = [ age,rooms,bedrooms,pop,households,income]

** Lege die Eingabefunktion für das Schätzerobjekt an. (Du kannst mit batch_size und num_epochs herumspielen)**

In [ ]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train ,batch_size=10,num_epochs=1000,
                                            shuffle=True)

** Erstelle das Schätzmodell. Verwenden einen DNNRegressor. Spiele mit den versteckten Einheiten rum. **

In [ ]:
model = tf.estimator.DNNRegressor(hidden_units=[6,6,6],feature_columns=feat_cols)

##### ** Trainiere das Modell für ~1.000 Schritte. (Komme später darauf zurück und trainiere  es für mehr und überprüfe es auf Verbesserungen. **

In [ ]:
model.train(input_fn=input_func,steps=25000)

** Erstelle eine Vorhersage-Eingabefunktion und verwende dann die Methode .predict aus Ihrem Schätzmodell, um eine Liste oder Vorhersagen für deine Testdaten zu erstellen. **

In [ ]:
predict_input_func = tf.estimator.inputs.pandas_input_fn(
      x=X_test,
      batch_size=10,
      num_epochs=1,
      shuffle=False)

In [ ]:
pred_gen = model.predict(predict_input_func)

In [ ]:
predictions = list(pred_gen)

** Berrechne den RMSE. Dies kannst du manuell machen oder das hier verwenden: [sklearn.metrics](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) **

In [ ]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['predictions'])

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
mean_squared_error(y_test,final_preds)**0.5

# Gut gemacht! 